In [88]:
from urllib.request import urlopen
#pip install bs4
from bs4 import BeautifulSoup
import requests

In [89]:
html=urlopen('https://lenta.ru/news/2020/09/27/bayrenlost/')
bs=BeautifulSoup(html.read(),'html.parser')
print(bs.h1)

<h1 class="b-topic__title" itemprop="headline">«Бавария» проиграла впервые с декабря прошлого года</h1>


In [90]:
class Content:
    def __init__(self, url,title, body):
        self.url = url
        self.title = title
        self.body = body
        
def get_page(url):
    req=requests.get(url)
    if req.status_code==200:
        return BeautifulSoup(req.text,'html.parser')
    return None

        

In [91]:
def news_from_lenta(url):
    bs=get_page(url)
    if bs is None:
        return bs
    title=bs.find("h1",{"class": "b-topic__title"}).text
    lines=bs.find_all("p")
    body='\n'.join([line.text for line in lines])
    return Content(url, title, body)
    #----

In [92]:
content=news_from_lenta('https://lenta.ru/articles/2018/03/22/slovenia')
if content is None:
    print('ошибка!')
else:
    print('Заголовок: {}'.format(content.title))
    print('Адрес: {}'.format(content.url))
    print(content.body)


Заголовок: Маршрут построен: Словения
Адрес: https://lenta.ru/articles/2018/03/22/slovenia
Всего 47 километров береговой линии — но у Словении они есть. Да, Словению можно рассматривать как пляжное направление. Причем по всем параметрам она ничуть не уступает своим соседям — Хорватии и Италии. «Лента.ру» выяснила, почему стоит обратить внимание на это не самое очевидное направление.
Как и вся Словения, приморский регион удивляет как минимум тем, что на такой маленькой территории будто мозаикой собраны красивые города со средневековой архитектурой, протяженные виноградники и оливковые рощи, соляные прииски, колоритные деревни и долины. Долгое время местность была под влиянием венецианцев — так что все побережье двуязычно, итальянский язык и словенский абсолютно равноправны. Архитектура тоже все время напоминает Италию, а некоторые здания и монументы и вовсе мысленно переносят в Венецию. При этом регион нисколько не теряет своего уникального шарма.
Заметный плюс отдыха на Словенской Ривь

In [93]:
def news_from_rt(url):
    bs=get_page(url)
    if bs is None:
        return bs
    title=bs.find("h1",{"class": "article__heading"}).text
   # title=bs.select("h1.article__heading").text
    lines=bs.find_all("div", {"class":"article__summary"})
    body='\n'.join([line.text for line in lines])
    return Content(url, title, body)
    #---

In [94]:
content=news_from_rt('https://russian.rt.com/science/article/504816-nauka-iskusstvo-slem')
if content is None:
    print('ошибка!')
else:
    print('Заголовок: {}'.format(content.title))
    print('Адрес: {}'.format(content.url))
    print(content.body)

Заголовок: «Без эстетики и красоты исследование невозможно»: что рассказали учёные о связи искусства и науки
Адрес: https://russian.rt.com/science/article/504816-nauka-iskusstvo-slem
В Центральном доме художника в Москве прошла «битва» учёных в формате стендапа, посвящённая искусству в науке, — Science Slam Art. За звание лучшего слэмера боролись четыре исследователя из МГУ им. М.В. Ломоносова. Учёные рассказали о красоте горных пород и жидких кристаллов, о взгляде современных урбанистов на мегаполис и визуализации данных из социальных сетей. Победителя определяла публика. Самые бурные аплодисменты сорвал доктор химических наук, профессор РАН Алексей Бобровский с рассказом о свойствах жидких кристаллов. Что успели за 10 минут сообщить учёные о связи искусства и науки, выяснял RT.


In [95]:
data=[
    ["https://russian.rt.com", "h1.article__heading", "div.article__summary"],
    ["https://lenta.ru", "h1.b-topic__title", "p"]
]


In [96]:
class Site:
     def __init__(self, host, title_tag, body_tag):
        self.url = host
        self.title = title_tag
        self.body = body_tag

sites =[]
for site in data:
    sites.append(Site(site[0], site[1], site[2]))

In [97]:
sites

[<__main__.Site at 0x7073870>, <__main__.Site at 0x7073bf0>]

In [98]:
class Content:
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body
        
    def show(self):
        print('Заголовок: {}'.format(self.title))
        print('Адрес: {}'.format(self.url))
        print(self.body)
        print()
        

In [101]:
class webBot:
    def get_page(self, url):
        try:
            req=requests.get(url)
        except:
            return None
        return BeautifulSoup(req.text, 'html.parser')
    
    def get_data(self, obj, selector):
        elements = obj.select(selector)
        if elements is not None and len(elements) > 0:
            return '\n'.join([element.text for element in elements])
        return ''
    
    def parse(self, site, url):
        full_path = site.url + url
        bs = get_page(full_path)
        if bs is not None:
            title = self.get_data(bs, site.title)
            body = self.get_data(bs, site.body)
            if title != '' and body !='':
                content = Content(full_path, title, body)
                content.show()
        

In [102]:
bot= webBot()
bot.parse(sites[0], '/science/article/504816-nauka-iskusstvo-slem')
bot.parse(sites[0], '/science/article/504897-mariinskie-gimnazii-zhenskoe-obrazovanie')
bot.parse(sites[0], '/science/article/504118-roboty-evroparlament-yuridicheskoye-lico')

Заголовок: «Без эстетики и красоты исследование невозможно»: что рассказали учёные о связи искусства и науки
Адрес: https://russian.rt.com/science/article/504816-nauka-iskusstvo-slem
В Центральном доме художника в Москве прошла «битва» учёных в формате стендапа, посвящённая искусству в науке, — Science Slam Art. За звание лучшего слэмера боролись четыре исследователя из МГУ им. М.В. Ломоносова. Учёные рассказали о красоте горных пород и жидких кристаллов, о взгляде современных урбанистов на мегаполис и визуализации данных из социальных сетей. Победителя определяла публика. Самые бурные аплодисменты сорвал доктор химических наук, профессор РАН Алексей Бобровский с рассказом о свойствах жидких кристаллов. Что успели за 10 минут сообщить учёные о связи искусства и науки, выяснял RT.

Заголовок: Женское дело: как Мариинские гимназии повлияли на развитие образования в России
Адрес: https://russian.rt.com/science/article/504897-mariinskie-gimnazii-zhenskoe-obrazovanie
160 лет назад в Санкт-П